# Bab 7: Ensemble Learning and Random Forests

### 1. Pendahuluan

Bab 7 membahas *Ensemble Learning*, sebuah teknik *Machine Learning* di mana Anda melatih beberapa prediktor (pengklasifikasi atau regressor) yang berbeda, kemudian menggabungkan prediksi mereka untuk mendapatkan hasil yang lebih baik. Metode *Ensemble* biasanya mengungguli model tunggal terbaik karena adanya "kebijaksanaan orang banyak" (*wisdom of the crowd*).

Bab ini akan mencakup beberapa algoritma *ensemble* populer seperti:
* Voting Classifiers
* Bagging dan Pasting
* Random Forests
* Boosting (AdaBoost & Gradient Boosting)
* Stacking

---

### 2. Voting Classifiers (Pengklasifikasi Voting)

Jika Anda melatih beberapa pengklasifikasi yang berbeda, Anda dapat mencapai akurasi yang lebih tinggi dengan menggabungkan prediksi mereka. Pengklasifikasi *voting* bekerja dengan mengumpulkan prediksi dari setiap pengklasifikasi.

* **Hard Voting (Mayoritas Langsung):** Memprediksi kelas yang menerima suara terbanyak.
* **Soft Voting (Mayoritas Probabilitas):** Memprediksi kelas dengan probabilitas kelas rata-rata tertinggi. Ini seringkali mengungguli *hard voting* karena memberikan bobot lebih pada prediksi dengan keyakinan tinggi.

Agar *ensemble* bekerja dengan baik, para pengklasifikasi haruslah beragam (membuat jenis kesalahan yang berbeda).


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Membuat dataset dan membaginya
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Inisialisasi beberapa model
log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(probability=True, random_state=42) # probability=True untuk soft voting

# Membuat Voting Classifier
# voting='hard' untuk mayoritas suara
# voting='soft' untuk rata-rata probabilitas
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft'
)

voting_clf.fit(X_train, y_train)

# Membandingkan akurasi setiap model
print("Akurasi setiap model:")
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

Akurasi setiap model:
LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.92


Biasanya, akurasi VotingClassifier akan sedikit lebih tinggi daripada model individual terbaik.

---

### 3. Bagging dan Pasting
Kedua strategi ini melibatkan pelatihan beberapa predictor dari jenis yang sama pada subset acak dari training set.

Bagging (Bootstrap Aggregation): Pengambilan sampel dilakukan dengan penggantian (with replacement).
Pasting: Pengambilan sampel dilakukan tanpa penggantian (without replacement).
Karena semua predictor dilatih secara paralel, Bagging/Pasting sangat cocok untuk pemrosesan paralel.

### a. Out-of-Bag (oob) Evaluation
Dengan Bagging, beberapa instance mungkin tidak pernah diambil sampelnya untuk pelatihan predictor tertentu. Instance ini disebut out-of-bag (oob) dan dapat digunakan sebagai validation set tanpa perlu memisahkan data.

In [2]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Membuat ensemble dengan 500 Decision Tree
# max_samples=100: setiap tree dilatih pada 100 instance acak
# bootstrap=True: sampling dengan pengembalian (bagging)
# oob_score=True: meminta evaluasi oob otomatis setelah training
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, oob_score=True, random_state=42)
bag_clf.fit(X_train, y_train)

# Skor oob adalah estimasi akurasi pada data yang tidak terlihat
print("OOB score:", bag_clf.oob_score_)

# Membandingkan dengan akurasi pada test set
y_pred = bag_clf.predict(X_test)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))

OOB score: 0.9253333333333333
Accuracy on test set: 0.904


Skor OOB seringkali menjadi estimasi yang baik untuk performa model pada test set.

---

### 4. Random Forests (Hutan Acak)
Random Forests adalah ensemble dari Decision Trees, yang dilatih melalui metode bagging. Keunikan utamanya adalah selain mengambil sampel instance secara acak, ia juga mengambil sampel fitur secara acak di setiap split node. Hal ini menghasilkan pohon yang lebih beragam dan model yang lebih kuat.

### a. Feature Importance (Pentingnya Fitur)
Random Forests menyediakan cara mudah untuk mengukur pentingnya relatif dari setiap fitur dengan melihat seberapa besar fitur tersebut mengurangi impurity rata-rata.

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# Inisialisasi Random Forest
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)
print("Random Forest accuracy on moons dataset:", accuracy_score(y_test, y_pred_rf))

# Menghitung feature importance pada dataset Iris
iris = load_iris()
rnd_clf_iris = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf_iris.fit(iris["data"], iris["target"])

print("\nFeature importances on Iris dataset:")
for name, score in zip(iris["feature_names"], rnd_clf_iris.feature_importances_):
    print(name, score)

Random Forest accuracy on moons dataset: 0.912

Feature importances on Iris dataset:
sepal length (cm) 0.11249225099876375
sepal width (cm) 0.02311928828251033
petal length (cm) 0.4410304643639577
petal width (cm) 0.4233579963547682


Feature importance menunjukkan fitur mana yang paling berpengaruh dalam membuat keputusan, yang sangat berguna untuk analisis.

---

### 5. Boosting
Boosting adalah teknik ensemble di mana predictor dilatih secara sekuensial, dengan setiap predictor baru mencoba untuk memperbaiki kesalahan dari predictor sebelumnya.

### a. AdaBoost (Adaptive Boosting)
Fokus pada instance yang salah diklasifikasikan oleh predictor sebelumnya dengan cara meningkatkan bobotnya.

In [4]:
from sklearn.ensemble import AdaBoostClassifier

# Menggunakan Decision Tree dangkal (stump) sebagai base estimator
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

y_pred_ada = ada_clf.predict(X_test)
print("\nAdaBoost accuracy:", accuracy_score(y_test, y_pred_ada))

c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



AdaBoost accuracy: 0.896


### b. Gradient Boosting
Melatih setiap predictor baru pada residual error (selisih antara target sebenarnya dan prediksi) dari predictor sebelumnya.

In [5]:
from sklearn.ensemble import GradientBoostingRegressor

# Membuat data acak
np.random.seed(42)
X_reg = 2 * np.random.rand(100, 1) - 1
y_reg = 3 * X_reg[:, 0]**2 + 0.05 * np.random.randn(100)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)
gbrt.fit(X_reg, y_reg)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

**XGBoost** (Extreme Gradient Boosting) adalah implementasi Gradient Boosting yang sangat dioptimalkan dan seringkali menjadi pilihan utama untuk data terstruktur karena performa dan kecepatannya.

---

### 6. Stacking (Stacked Generalization)
Stacking adalah metode ensemble di mana sebuah model akhir (disebut blender atau meta-learner) dilatih untuk melakukan agregasi (menggabungkan) prediksi dari semua predictor di dalam ensemble.

Prosesnya secara umum adalah:

1. Dataset dibagi menjadi beberapa bagian.
2. Beberapa model (base learners) dilatih pada bagian pertama.
3. Model-model tersebut membuat prediksi pada bagian kedua.
4. Prediksi-prediksi ini digunakan sebagai fitur baru untuk melatih meta-learner.
5. Meta-learner inilah yang akan membuat prediksi akhir.